# EXTERNAL

In [1]:
#@title OpenBioLink Module Install EXTERNAL{ form-width: "15%" }
! pip install openbiolink

     |████████████████████████████████| 226 kB 12.5 MB/s 


In [2]:
#@title PyG 113 Installation EXTERNAL{ form-width: "15%" }
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 9.8 MB/s 
     |████████████████████████████████| 3.5 MB 10.3 MB/s 
     |████████████████████████████████| 407 kB 10.8 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 


# HEADER

In [19]:
#@title Module Imports HEADER { form-width: "15%" }
from openbiolink.obl2021 import OBL2021Dataset, OBL2021Evaluator
import torch
from torch.nn import Module,\
                     ModuleList,\
                     Embedding,\
                     BatchNorm1d,\
                     Linear,\
                      BCEWithLogitsLoss,\
                     CrossEntropyLoss,\
                     Dropout
from torch.optim import Adam
import torch.nn.functional as F
import torch_geometric as PyG
from torch_geometric.data import Data
from torch_geometric.nn.conv import RGCNConv
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from typing import NoReturn
import typing
import time
from enum import Enum
from datetime import datetime
from collections import defaultdict

# GLOBAL 
<!-- ###now CPU -->

In [27]:
#@title Global Variables CLASS{ form-width: "15%" }
class Global(Enum):
  HEAD_INDEX = 0
  RELATION_INDEX = 1
  TAIL_INDEX = 2
  FEATURE_ENG = 'one-hot'
  NUM_RELATIONS = 28
  DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
  MINI_BATCH_SIZE = 32
  DISP_HITS = True
  MANUAL_SEED = 2021


torch.manual_seed(Global.MANUAL_SEED.value)
dataset = OBL2021Dataset()
evaluator = OBL2021Evaluator()
kg = torch.cat((dataset.training, dataset.validation, dataset.testing), dim=0)
sorted_on_tails_indcs_kg = torch.sort(kg[:, Global.TAIL_INDEX.value])[1]
kg_sorted_tails = kg[sorted_on_tails_indcs_kg]
sorted_on_tails_indcs_kg = None
sorted_on_heads_indcs_kg = torch.sort(kg[:, Global.HEAD_INDEX.value])[1]
kg_sorted_heads = kg[sorted_on_heads_indcs_kg]
sorted_on_heads_indcs_kg = None
features = dataset.candidates.to(Global.DEVICE.value) 
train_set = dataset.training.to(Global.DEVICE.value) # torch.tensor of shape(num_train,3)
val_set = dataset.validation.to(Global.DEVICE.value) # torch.tensor of shape(num_val,3)
test_set = dataset.testing.to(Global.DEVICE.value)   # torch.tensor of shape(num_train,3)
dataset = None
sorted_train_set = train_set[torch.sort(train_set[:, 0])[1]]

Dataset found in /content/obl2021, omitting download...


# GNN Model and Hyperparameters

In [28]:
#@title GNN Model CLASS{ form-width: "10%" }
class GNN(Module):
  def __init__(self, conv_dims: list, fully_connected_for_head_dims: list, fully_connected_for_tail_dims: list, x_feature:str, dropout: dict, embedding_dims: tuple=None)-> NoReturn:
    super(GNN, self).__init__()
    self.conv_dropout = Dropout(p=dropout["conv"])
    self.emb_dropout = Dropout(p=dropout["emb"]) 
    self.fc_dropout = Dropout(p=dropout["fc"])
    self.x_feature = x_feature
    self.num_relations = Global.NUM_RELATIONS.value
    self.relation_embedder = Embedding(self.num_relations, fully_connected_for_tail_dims[-1])
    if x_feature == 'one-hot':
      self.entity_embedder = Embedding(embedding_dims[0], embedding_dims[1])
      first_conv_layer = [RGCNConv(embedding_dims[1], conv_dims[0], self.num_relations)]
    elif x_feature == 'identity':
      first_conv_layer = [RGCNConv(1, conv_dims[0], self.num_relations)]
    conv_list = first_conv_layer + \
                                [
                                  RGCNConv(conv_dims[i], conv_dims[i+1], self.num_relations)
                                  for i in range(len(conv_dims[:-1]))
                                ]
  
    fully_connected_list_for_tail = [
                                      Linear(conv_dims[-1], fully_connected_for_tail_dims[0]) 
                                    ] +\
                                    [
                                     Linear(fully_connected_for_tail_dims[i], fully_connected_for_tail_dims[i + 1])
                                     for i in range(len(fully_connected_for_tail_dims[:-1]))
                                    ]

    fully_connected_list_for_head = [
                                      Linear(conv_dims[-1], fully_connected_for_head_dims[0]) 
                                    ] +\
                                    [
                                     Linear(fully_connected_for_head_dims[i], fully_connected_for_head_dims[i + 1])
                                     for i in range(len(fully_connected_for_head_dims[:-1]))
                                    ]
    

    #graph conv layers
    self.conv_layers = ModuleList(conv_list)

    #fully connected dense layers
    self.head_fc_layers = ModuleList(fully_connected_list_for_head)
    self.tail_fc_layers = ModuleList(fully_connected_list_for_tail)
    

  def reset_parameters(self):
    self.entity_embedder.reset_parameters()
    self.relation_embedder.reset_parameters()
    for conv in self.conv_layers:
        conv.reset_parameters()
    for fc in self.head_fc_layers:
        fc.reset_parameters()
    for fc in self.tail_fc_layers:
        fc.reset_parameters()


  def forward(self, data: Data) -> torch.Tensor:
    edge_index = data.edge_index_messaging
    x = data.x
    edge_type = data.edge_type_messaging

    ####################################### ONE - HOT #######################################
    if self.x_feature == 'one-hot':
      x = self.entity_embedder(x).reshape(self.entity_embedder.weight.shape[0], -1)
      
    ############################################## IDENTITY ################################################
    elif self.x_feature == 'identity':
      x = torch.ones(x.shape[0], 1) ############
      if self.training:
        x = self.emb_dropout(x)
    ####################################### RGCN Encoder #######################################
    for conv in self.conv_layers[:-1]:
      x = conv(x, edge_index=edge_index, edge_type=edge_type)
      x = F.relu(x)
      if self.training:
        x = self.conv_dropout(x)
    x = self.conv_layers[-1](x, edge_index, edge_type)
    if self.training:
      x = self.conv_dropout(x)
    
    ####################################### Decoder #######################################

    positive_heads = x[data.edge_index_supervision[0]]
    positive_tails = x[data.edge_index_supervision[1]]
    positive_relations = self.relation_embedder(data.edge_type_supervision)

    negative_heads = x[data.edge_index_negative[0]]
    negative_tails = x[data.edge_index_negative[1]]
    negative_relations = self.relation_embedder(data.edge_type_negative)
    if self.training:
      negative_relations = self.emb_dropout(negative_relations)
      positive_relations = self.emb_dropout(positive_relations)

    for head_fc, tail_fc in zip(self.head_fc_layers, self.tail_fc_layers):
      positive_heads = head_fc(positive_heads)
      positive_heads = F.relu(positive_heads)

      negative_heads = head_fc(negative_heads)
      negative_heads = F.relu(negative_heads)
      
      positive_tails = tail_fc(positive_tails)
      positive_tails = F.relu(positive_tails) 

      negative_tails = tail_fc(negative_tails)
      negative_tails = F.relu(negative_tails)

      if self.training:
        positive_heads = self.conv_dropout(positive_heads)
        negative_heads = self.conv_dropout(negative_heads)
        positive_tails = self.conv_dropout(positive_tails)
        negative_tails = self.conv_dropout(negative_tails)
    
    # let f be the fully connected transform function for heads, and 
    # let g be the fully connected transform function for tails:
        # prior to the fully connected transforms:
        #     score(head, relation, tail) = <head, relation, tail> = head . relation . tail
        #     score(tail, relation, head) = <tail, relation, head> = tail . relation . head
        #     ===> score(head, relation, tail) = score(tail, relation, head) essentially holds
        # now with the fully connected transforms:
        #     score(head, relation, tail) = <f(head), relation, g(tail)> = f(head) . relation . g(tail)
        #     score(tail, relation, head) = <f(tail), relation, g(head)> = f(tail) . relation . g(head)
        #     ===> score(head, relation, tail) = score(tail, relation, head) does not necessarily holds

    return (positive_heads * positive_relations * positive_tails).sum(dim=1), \
           (negative_heads * negative_relations * negative_tails).sum(dim=1)

In [29]:
#@title Model and Hyperparameters MAIN { form-width: "15%" }
model = GNN(
    x_feature='one-hot',
    conv_dims=[32, 64, 64, 64],
    embedding_dims=(features.max() + 1, 32),
    fully_connected_for_head_dims=[64, 32], 
    fully_connected_for_tail_dims=[64, 32],
    dropout={
        "emb": 0.1,
        "conv": 0.2,
        "fc": 0.3
    }
).to(Global.DEVICE.value)
print(model)
loss_fn = torch.nn.BCEWithLogitsLoss()
opt = Adam(model.parameters())

GNN(
  (conv_dropout): Dropout(p=0.2, inplace=False)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (fc_dropout): Dropout(p=0.3, inplace=False)
  (relation_embedder): Embedding(28, 32)
  (entity_embedder): Embedding(180992, 32)
  (conv_layers): ModuleList(
    (0): RGCNConv(32, 32, num_relations=28)
    (1): RGCNConv(32, 64, num_relations=28)
    (2): RGCNConv(64, 64, num_relations=28)
    (3): RGCNConv(64, 64, num_relations=28)
  )
  (head_fc_layers): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=32, bias=True)
  )
  (tail_fc_layers): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=32, bias=True)
  )
)


#FUNCTIONS

In [30]:
#@title find_start_and_end_indcs() PRIVATE-FUNCTION { form-width: "15%" }
def find_start_and_end_indcs(entity, entity_type, kg_sorted):
  up = kg_sorted.shape[0]
  down = 0
  indx = kg_sorted.shape[0] // 2
  found = False
  while up - down > 1:
    if kg_sorted[indx][entity_type].item() == entity:
      found = True
      break 
    elif kg_sorted[indx][entity_type].item() >= entity: 
      up = indx
      indx = (up + down) // 2
    else:
      down = indx 
      indx = (up + down) // 2
  if not found:
    return None
  while 1:
    indx += 1
    try:
      if not kg_sorted[indx][entity_type].item() == entity:
        indx -= 1
        end_indx = indx
        break
    except:
      end_indx = indx - 1
      break
  

  while 1:
    indx -= 1
    try:
      if not kg_sorted[indx][entity_type].item() == entity:
        indx += 1
        start_indx = indx
        break
    except:
      start_indx = indx + 1
      break
  return torch.tensor(range(start_indx, end_indx + 1))

In [31]:
#@title check_negative_samples() FUNCTION { form-width: "15%" }
def check_negative_samples(negative_samples, sorted_train_set):
  for triple in negative_samples:
    indcs = find_start_and_end_indcs(triple[0], 0, sorted_train_set)
    if isinstance(indcs, torch.Tensor):
      facts = sorted_train_set[indcs]
      are_actually_corrupt = ((triple == facts).type(torch.int8).sum(dim=-1) < 3).sum()
      if not are_actually_corrupt.type(torch.int8):
        return False 
  return True

In [32]:
#@title get_psuedo_negative_entities() FUNCTIONS { form-width: "15%" }
def get_psuedo_negative_entities(entity, corrupt_at, kg_sorted):
  if corrupt_at == Global.HEAD_INDEX.value:
    entity_type = Global.TAIL_INDEX.value 
  elif corrupt_at == Global.TAIL_INDEX.value:
    entity_type = Global.HEAD_INDEX.value

  indcs = find_start_and_end_indcs(entity, entity_type, kg_sorted=kg_sorted)
  
  if indcs is not None:
    fact_triplets_entities = kg_sorted[indcs][:, corrupt_at]
    features_copy = features.detach().clone()
    features_copy[fact_triplets_entities] = -1
    non_negative_mask = features_copy >= 0
    ret = torch.nonzero(non_negative_mask).reshape(-1)
    return ret
  else:
    return features.to(Global.DEVICE.value)

In [33]:

#@title #TODO NEGATIVE SAMPLING use MSCaching FUNCTION  { form-width: "15%" }

def mini_batch_maker(messaging, supervision, candidates, x_feature='one-hot'):
  # heads = supervision[:, 0]
  # relations = supervision[:, 1]
  # tails = supervision[:, -1]

  # ct_size = supervision.shape[0] // 2
  # ch_size = supervision.shape[0] - ct_size

  # negative_samples_corrupted_tails = torch.vstack(
  #     (
  #         heads[: ct_size], 
  #         relations[: ct_size], 
  #         torch.multinomial(
  #             candidates.type(torch.float).to(Global.DEVICE.value), 
  #             ct_size
  #         )
  #     )
  # ).t().contiguous()

  # negative_samples_corrupted_heads = torch.vstack(
  #     ( 
  #         torch.multinomial(
  #             candidates.type(torch.float).to(Global.DEVICE.value), 
  #             ch_size
  #         ), 
  #         relations[ch_size:], 
  #         tails[ch_size:]
  #     )
  # ).t().contiguous()

  # negative_samples = torch.cat(
  #     (negative_samples_corrupted_heads, negative_samples_corrupted_tails),
  #     dim=0
  # )
  relations = supervision[:, 1]
  while True:
    batch_size = supervision.shape[0]
    negative_samples = torch.vstack(
        (
            torch.multinomial(candidates.type(torch.float).to(Global.DEVICE.value), batch_size),
            relations,
            torch.multinomial(candidates.type(torch.float).to(Global.DEVICE.value), batch_size)
        )
    ).t().contiguous()
    if check_negative_samples(negative_samples, sorted_train_set):
      break

  graph = graph_data_maker(
      messaging=messaging,
      supervision=supervision,
      negative_samples=negative_samples,
      x=candidates.to(Global.DEVICE.value),
      x_feature='one-hot'
  )

  return graph

In [34]:
#@title graph_data_maker(dataset, x_feature) FUNCTION{ form-width: "15%" }
def graph_data_maker(messaging: torch.Tensor, supervision: torch.Tensor, negative_samples: torch.Tensor, x:torch.Tensor, x_feature: str=Global.FEATURE_ENG.value, check_for_correctness: bool=False) -> Data:
  relation_idx = Global.RELATION_INDEX.value
  head_idx = Global.HEAD_INDEX.value
  tail_idx = Global.TAIL_INDEX.value
  graph_data = Data(
        x=x.reshape(-1, 1),
        edge_index_messaging=messaging[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_messaging=messaging[:, relation_idx],
        edge_index_supervision=supervision[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_supervision=supervision[:, relation_idx],
        edge_index_negative=negative_samples[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_negative=negative_samples[:, (relation_idx)]
    )
  return graph_data


In [35]:
#@title train(model, graph, optimizer, loss_fn) FUNCTION{ form-width: "15%" }
def train(model: GNN, graph: Data, optimizer: torch.optim, loss_fn:torch.nn.modules.loss) -> tuple((torch.float, torch.Tensor, torch.Tensor)):
  optimizer.zero_grad()
  model.train()
  positive, negative = model(graph)
  loss = loss_fn(
            negative, 
            torch.zeros(graph.edge_index_negative.shape[1]).to(Global.DEVICE.value)
         ) + loss_fn(
                positive,
                torch.ones(graph.edge_index_supervision.shape[1]).to(Global.DEVICE.value)
            )
  loss.backward()
  optimizer.step()
  return loss.item(), torch.sigmoid(positive), torch.sigmoid(negative)

In [36]:
#@title evaluation_rank() PRIVATE-FUNCTION { form-width: "10%" }
@torch.no_grad()
def evaluation_rank(model, eval_set, messaging_set):
  h_pred_top10 = list()
  t_pred_top10 = list()

  for eval_triplet in tqdm(eval_set):
    head = eval_triplet[Global.HEAD_INDEX.value].item()
    relation = eval_triplet[Global.RELATION_INDEX.value].item()
    tail = eval_triplet[Global.TAIL_INDEX.value].item()

    corrupted_tails = get_psuedo_negative_entities(
        entity=head, 
        corrupt_at=Global.TAIL_INDEX.value, 
        kg_sorted=kg_sorted_heads
    )
    
    num_psuedo_negative_triples = corrupted_tails.shape[0]
    psuedo_negative_triplets_corrupted_tail = torch.vstack(
        (
            torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * head,
            torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * relation,
            corrupted_tails
        )
    ).t().to(Global.DEVICE.value)

    corrupted_heads = get_psuedo_negative_entities(
        entity=tail, 
        corrupt_at=Global.HEAD_INDEX.value, 
        kg_sorted=kg_sorted_tails
    )

    num_psuedo_negative_triples = corrupted_heads.shape[0]
    psuedo_negative_triplets_corrupted_head = torch.vstack(
        (
            corrupted_heads,
            torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * relation,
            torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * tail
        )
    ).t().to(Global.DEVICE.value)

    # eval_triplet: (h, r, t)
    # psuedo_negative_triplets_head: (h′, r, t) for all h′
    # psuedo_negative_triplets_tail: (h, r, t′) for all t′

    # train_set being the messaging graph, calculate the score for (h, r, t)
    # train_set being the messaging graph, calculate the scores for all (h′, r, t)
    # train_set being the messaging graph, calculate the scores for all (h, r, t′)

    graph_data_for_object_tail = graph_data_maker(
      messaging=messaging_set,
      supervision=eval_triplet.reshape(1, 3),
      negative_samples=psuedo_negative_triplets_corrupted_head,
      x=features.to(Global.DEVICE.value),
      x_feature='one-hot'
    )

    graph_data_for_object_head = graph_data_maker(
            messaging=messaging_set,
            supervision=eval_triplet.reshape(1, 3),
            negative_samples=psuedo_negative_triplets_corrupted_tail,
            x=features.to(Global.DEVICE.value),
            x_feature='one-hot'
    )

    model.eval()
    scores_for_object_tail = torch.cat(model(graph_data_for_object_tail))

    sorted_by_scores_for_object_tail_indcs = torch.sort(scores_for_object_tail, descending=True)[1]

    head_objects = torch.cat((graph_data_for_object_tail.edge_index_supervision[0], graph_data_for_object_tail.edge_index_negative[0]))
    top10_heads = head_objects[sorted_by_scores_for_object_tail_indcs[:10]]
    h_pred_top10.append(top10_heads)

    model.eval()
    scores_for_object_head = torch.cat(model(graph_data_for_object_head))

    sorted_by_scores_for_object_head_indcs = torch.sort(scores_for_object_head, descending=True)[1]

    tail_objects = torch.cat((graph_data_for_object_head.edge_index_supervision[1], graph_data_for_object_head.edge_index_negative[1]))
    top10_tails = tail_objects[sorted_by_scores_for_object_head_indcs[:10]]
    t_pred_top10.append(top10_tails)

    print('')
    ############################################################################## 1
    if Global.DISP_HITS.value:
      hit = False
      hit_at10 = eval_triplet[2] in tail_objects[sorted_by_scores_for_object_head_indcs[:10]]
      if hit_at10:
        print(f'Tail hit @ 10')
        hit = True
      hit_at50 = eval_triplet[2] in tail_objects[sorted_by_scores_for_object_head_indcs[:50]]
      if not hit and hit_at50:
        print(f'Tail hit @ 50')
        hit = True
      hit_at100 = eval_triplet[2] in tail_objects[sorted_by_scores_for_object_head_indcs[:100]]
      if not hit and hit_at100:
        print(f'Tail hit @ 100')
        hit = True
      hit_at1000 = eval_triplet[2] in tail_objects[sorted_by_scores_for_object_head_indcs[:1000]]
      if not hit and hit_at1000:
        print(f'Tail hit @ 1000')
        hit = True
      if hit:
        print('-' * 30)
    ############################################################################### 2

    ############################################################################### 3
    hit = False
    if Global.DISP_HITS.value:
      hit_at10 = eval_triplet[0] in head_objects[sorted_by_scores_for_object_tail_indcs[:10]]
      if hit_at10:
        print('Head hit @ 10')
        hit = True
      hit_at50 = eval_triplet[0] in head_objects[sorted_by_scores_for_object_tail_indcs[:50]]
      if not hit and hit_at50:
        print(f'Head hit @ 50')
      hit_at100 = eval_triplet[0] in head_objects[sorted_by_scores_for_object_tail_indcs[:100]]
      if not hit and hit_at100:
        print(f'Head hit @ 100')
      hit_at1000 = eval_triplet[0] in head_objects[sorted_by_scores_for_object_tail_indcs[:1000]]
      if not hit and hit_at1000:
        print(f'Head hit @ 1000')
        hit = True
      if hit:
        print('-' * 30)
    ############################################################################## 4
    
  model.train()
  return torch.stack(h_pred_top10), torch.stack(t_pred_top10)

In [37]:
#@title evaluate_hits_at_10(model, mode) FUNCTION{ form-width: "15%" }
def evaluate_hits_at_10(model: GNN, mode:str) -> torch.float:
  if mode == 'validation':
    head_objects_top10, tail_objects_top10 = evaluation_rank(model, val_set, train_set)
    return evaluator.eval(
            head_objects_top10,
            tail_objects_top10,
            val_set,
            False
          )
  elif mode == 'testing':
    head_objects_top10, tail_objects_top10 = evaluation_rank(model, test_set, torch.cat((train_set, val_set), dim=0))
    return evaluator.eval(
            head_objects_top10,
            tail_objects_top10,
            test_set,
            False
          )


In [52]:
#@title save_execution() FUNCTION { form-width: "15%" }
def save_execution(model, iteration, epoch, epoch_loss, download):
  from google.colab import files
  saved_time = str(datetime.now())
  torch.save(model, f'model@{saved_time}.pth')
  with open(f'info@{saved_time}.txt', 'w') as info_file:
    info_file.write(f'model: {str(model)}')
    info_file.write('\n')
    info_file.write(f'iteration: {iteration}\n')
    info_file.write(f'epoch: {epoch + 1}\n')
    info_file.write(f'epoch loss: {epoch_loss}')
  if download:
    files.download(f'info@{saved_time}.txt')
    files.download(f'model@{saved_time}.pth')

#Trainer

In [53]:
# @title Parameter and Hyperparameter Tuning MAIN{ form-width: "10%" }
model.reset_parameters()
try:
  for epoch in range(5):
    iteration = 0
    epoch_loss = 0
  ############################## TRAIN Graph Maker #####################################
    for i in tqdm(range(0, train_set.shape[0] - Global.MINI_BATCH_SIZE.value, Global.MINI_BATCH_SIZE.value)):
      iteration += 1
      train_supervision = train_set[i: i + Global.MINI_BATCH_SIZE.value, :]
      train_messaging = torch.cat(
          (train_set[: i, :], train_set[i + Global.MINI_BATCH_SIZE.value: , :]),
          dim=0
      )
      train_graph = mini_batch_maker(
          train_messaging,
          train_supervision,
          features,
          'one-hot'
      )
      loss, positive_score, negative_score = train(model, train_graph, opt, loss_fn)
      epoch_loss += loss
      if iteration % 100 == 0:
        print('')
        print('-' * 50)
        agg_p_score = positive_score.sum().item()
        agg_n_score = negative_score.sum().item()
        print(f'Train Batch {iteration}:')
        print('-' * 50)
        print(f'Batch Loss:    {loss: .4f}')
        print('-' * 50)
        print(f'Average Loss:         <{epoch_loss / iteration: .4f} >')
        print('-' * 50)
        print(f'Agg P-Score:   {agg_p_score: .4f}')
        print('-' * 50)
        print(f'Agg N-Score:   {agg_n_score: .4f}')
        print(f'===' * 25)
      if iteration % 1500 == 0:
        save_execution(model, iteration, epoch, epoch_loss, download=True)
except KeyboardInterrupt:
  save_execution(model, iteration, epoch, epoch_loss, download=False)


  0%|          | 2/131000 [00:03<66:48:41,  1.84s/it]


# TEST EVALUATION

In [43]:
a, b = evaluation_rank(model, val_set[:100], train_set)
evaluator.eval(
            a,
            b,
            val_set[:100],
            False
          )

  1%|          | 1/100 [00:01<02:18,  1.39s/it]


Head hit @ 1000
------------------------------


  2%|▏         | 2/100 [00:02<02:11,  1.34s/it]

  3%|▎         | 3/100 [00:03<02:08,  1.32s/it]

  4%|▍         | 4/100 [00:05<02:02,  1.27s/it]

...

 33%|███▎      | 33/100 [00:40<01:21,  1.22s/it]

 34%|███▍      | 34/100 [00:42<01:22,  1.24s/it]


Head hit @ 1000
------------------------------


 34%|███▍      | 34/100 [00:42<01:22,  1.26s/it]


KeyboardInterrupt: ignored

In [ ]:
try:
  evaluate_hits_at_10(model, 'testing')
except KeyboardInterrupt:
  pass

# ADD-ONS


In [ ]:
#@title add_tail_to_head(data_split) FUNCTION { form-width: "15%" }
def add_tail_to_head(data_split: torch.Tensor, plus: int=28) -> torch.Tensor:
  heads = data_split[:, (0)]
  tails = data_split[:, (-1)]
  relations = data_split[:, (1)]
  
  tail_to_head = torch.vstack(
      (tails, relations + plus, heads)
  ).t()

  return torch.cat(
      (data_split, tail_to_head),
      dim=0
  )



In [ ]:
#@title checksum(data_subset, graph_data) FUNCTION { form-width: "15%" }
def checksum(data_subset: torch.Tensor, graph_data: Data, step: int=500) -> NoReturn:
  for i in range(step, data_subset.shape[0], step):
    graph_head_indcs = graph_data.edge_index[0, i-step:i]
    data_head_entities = data_subset[i-step:i, 0]
    graph_head_entities = graph_data.x[graph_head_indcs].reshape(-1)

    checksum_head = (graph_head_entities == data_head_entities).sum().item()
    if not checksum_head == step:
      print('head')
      print(i)
      break

    graph_relation_indcs = graph_data.edge_type[i-step:i]
    data_relation_types = data_subset[i-step:i, 1]

    checksum_relation = (graph_relation_indcs == data_relation_types).sum().item()
    if not checksum_relation == step:
      print('relation')
      print(i)
      break

    graph_tail_indcs = graph_data.edge_index[1, i-step:i]
    data_tail_entities = data_subset[i-step:i, 2]
    graph_tail_entities = graph_data.x[graph_tail_indcs].reshape(-1)

    checksum_tail = (graph_tail_entities == data_tail_entities).sum().item()
    if not checksum_head == step:
      print('tail')
      print(i)
      break

  else:
    print('All clear :)')

In [ ]:
#@title partial_graph_data_maker(dataset, x_feature) FUNCTION{ form-width: "15%" }
def partial_graph_data_maker(messaging: torch.Tensor, supervision: torch.Tensor, negative_samples: torch.Tensor, x_feature: str=Global.FEATURE_ENG.value, check_for_correctness: bool=False) -> Data:
  relation_idx = Global.RELATION_INDEX.value
  head_idx = Global.HEAD_INDEX.value
  tail_idx = Global.TAIL_INDEX.value


  if x_feature == 'one-hot':
    one_hot_index = 0
    edge_index_list_flat = []
    seen_dict = defaultdict(lambda: -1)
    edge_index_flat = messaging[:, (head_idx, tail_idx)].reshape(-1)

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_messaging = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    edge_index_flat = supervision[:, (head_idx, tail_idx)].reshape(-1)
    edge_index_list_flat = []

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_supervision = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    edge_index_flat = negative_samples[:, (head_idx, tail_idx)].reshape(-1)
    edge_index_list_flat = []

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_negative = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    x = torch.tensor(list(seen_dict.keys())).reshape(-1, 1)

    graph_data = Data(
        x=x,
        edge_index_messaging=edge_index_messaging.t().contiguous(),
        edge_type_messaging=messaging[:, relation_idx],
        edge_index_supervision=edge_index_supervision.t().contiguous(),
        edge_type_supervision=supervision[:, relation_idx],
        edge_index_negative=edge_index_negative.t().contiguous(),
        edge_type_negative=negative_samples[:, (relation_idx)]
    )
  else:
    raise NotImplementedError('This functionality has not been implemented yet.')

  if check_for_correctness:
    checksum(dataset, graph_data)
  return graph_data

In [ ]:
#@title visualize_graph(graph_data, height, width) FUNCTION{ form-width: "15%" }
def visualize_graph(graph_data: Data, height: int=10, width:int=10) -> NoReturn:
  nx_graph = to_networkx(graph_data)

  pos = nx.spring_layout(nx_graph)

  fig = plt.gcf()
  fig.set_size_inches(width, height)

  edge_labels = dict()
  ei_np = graph_data.edge_index.t().numpy()
  for edge in nx_graph.edges():
    e = np.array(edge)
    idx = np.where(ei_np == e)[0][0]
    label = graph_data.edge_type[idx].item()
    edge_labels.update({edge: label})
    
  nx.draw_networkx_nodes(nx_graph, pos)
  nx.draw_networkx_edges(nx_graph, pos, connectionstyle='arc3,rad=0.2')
  nx.draw_networkx_labels(nx_graph, pos, labels={n:graph_data.x[n].item() for n in nx_graph})
  nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels=edge_labels)

  fig.show()


In [ ]:
#@title Visualization OPTIONAL-MAIN { form-width: "15%" }
data_subset = train_set[:2500, :]
graph_data = graph_data_maker(data_subset, 'one-hot')
visualize_graph(graph_data, 100, 100)